In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [16]:
data_transforms = transforms.Compose([
        transforms.Resize([256, 256]),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [41]:
data_dir = 'Data_for_test'
BATCH_SIZE = 16
image_dataset = datasets.ImageFolder(data_dir, transform=data_transforms)
train_dataset, val_dataset = torch.utils.data.random_split(image_dataset, [210, 90])
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_data = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
class_names = image_dataset.classes
dataset_sizes = {'train': 210, 'val': 90}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)

['Bed', 'Chair', 'Sofa']


In [43]:
dataloaders = {'train': train_data, 'val': val_data}

In [55]:
for inputs, labels in dataloaders['train']:
    inputs = inputs.to(device)
    labels = labels.to(device)
    print(inputs[0].shape)
    print(labels[0].item())
    break

torch.Size([3, 224, 224])
1


In [44]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    start = time.time()

    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - start
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model)
    return model

In [34]:
model_resnet = models.resnet50(pretrained=True)
for param in model_resnet.parameters():
    param.requires_grad = False

num_fc = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_fc, 3)

model_resnet = model_resnet.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model_resnet.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [45]:
model_resnet = train_model(model_resnet, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6812 Acc: 0.8095
val Loss: 0.5154 Acc: 0.8444
Epoch 1/24
----------
train Loss: 0.5717 Acc: 0.7810
val Loss: 0.4949 Acc: 0.8889
Epoch 2/24
----------
train Loss: 0.4381 Acc: 0.8952
val Loss: 0.2631 Acc: 0.9444
Epoch 3/24
----------
train Loss: 0.3111 Acc: 0.9000
val Loss: 0.2333 Acc: 0.9667
Epoch 4/24
----------
train Loss: 0.2773 Acc: 0.9476
val Loss: 0.2433 Acc: 0.9444
Epoch 5/24
----------
train Loss: 0.2888 Acc: 0.9333
val Loss: 0.2320 Acc: 0.9667
Epoch 6/24
----------
train Loss: 0.2677 Acc: 0.9429
val Loss: 0.2290 Acc: 0.9556
Epoch 7/24
----------
train Loss: 0.2624 Acc: 0.9381
val Loss: 0.2379 Acc: 0.9222
Epoch 8/24
----------
train Loss: 0.2756 Acc: 0.9286
val Loss: 0.2255 Acc: 0.9667
Epoch 9/24
----------
train Loss: 0.2539 Acc: 0.9476
val Loss: 0.2234 Acc: 0.9556
Epoch 10/24
----------
train Loss: 0.2672 Acc: 0.9381
val Loss: 0.2239 Acc: 0.9556
Epoch 11/24
----------
train Loss: 0.2516 Acc: 0.9571
val Loss: 0.2338 Acc: 0.9333
Epoch 12/24
--

In [49]:
torch.save(model_resnet.state_dict(), 'model_resnet50.pt')